In [ ]:
import numpy as np
import xarray as xr
import scipy.stats as st
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import datetime as dt
import pandas as pd
import time
import urllib.request
import metpy.calc as mpcalc
import salem
import scipy.optimize as opt
import warnings
import geopy.distance
import matplotlib as mpl

from scipy.ndimage.measurements import label
from scipy.ndimage import binary_dilation

warnings.filterwarnings('ignore')

dir_data='/Users/ahenny/'
dir2='/Volumes/Extreme Pro/'
dir3='/Volumes/My Passport/'

In [ ]:
ds=xr.open_dataset(dir2+'merra2.ar.labels.model.variable.'+str(2023)+'.nc')
lons=ds.lon.values
lats=ds.lat.values

In [ ]:
yrs=np.arange(1980,2024,1)

for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    
    ds=xr.open_dataset(dir2+'jra55.ar.labels.model.variable.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    ar=labeled/labeled
    freq_jra55_polar=ar.sum(dim='time',skipna=True)
    
    ds=xr.open_dataset(dir_data+'jra55.ar.labels.model.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    ar=labeled/labeled
    freq_jra55_fixed=ar.sum(dim='time',skipna=True)
    
    ds=xr.open_dataset(dir2+'jra55.ar.labels.model.poleward.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    ar=labeled/labeled
    freq_jra55_fixed_poleward=ar.sum(dim='time',skipna=True)
    
    if i==0:
        freq_concat_jra55_polar=freq_jra55_polar
        freq_concat_jra55_fixed=freq_jra55_fixed
        freq_concat_jra55_fixed_poleward=freq_jra55_fixed_poleward
    else:
        freq_concat_jra55_polar=xr.concat([freq_concat_jra55_polar,freq_jra55_polar],dim='time')
        freq_concat_jra55_fixed=xr.concat([freq_concat_jra55_fixed,freq_jra55_fixed],dim='time')
        freq_concat_jra55_fixed_poleward=xr.concat([freq_concat_jra55_fixed_poleward,freq_jra55_fixed_poleward],dim='time')
    
    print(freq_concat_jra55_polar.mean().values)
    print(freq_jra55_polar.mean().values)
    
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    
    ds=xr.open_dataset(dir2+'merra2.ar.labels.model.variable.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    labeled['lon']=lons
    labeled['lat']=lats
    ar=labeled/labeled
    freq_merra2_polar=ar.sum(dim='time',skipna=True)

    ds=xr.open_dataset(dir2+'merra2.ar.labels.model.'+str(year)+'.nc')
    labeled=ds['ar_labeled'][0::2,:,:]
    labeled['lon']=lons
    labeled['lat']=lats
    ar=labeled/labeled
    freq_merra2_fixed=ar.sum(dim='time',skipna=True)
    
    ds=xr.open_dataset(dir2+'merra2.ar.labels.model.poleward.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    labeled['lon']=lons
    labeled['lat']=lats
    ar=labeled/labeled
    freq_merra2_fixed_poleward=ar.sum(dim='time',skipna=True)
    
    if i==0:
        freq_concat_merra2_polar=freq_merra2_polar
        freq_concat_merra2_fixed=freq_merra2_fixed
        freq_concat_merra2_fixed_poleward=freq_merra2_fixed_poleward
    else:
        freq_concat_merra2_polar=xr.concat([freq_concat_merra2_polar,freq_merra2_polar],dim='time')
        freq_concat_merra2_fixed=xr.concat([freq_concat_merra2_fixed,freq_merra2_fixed],dim='time')
        freq_concat_merra2_fixed_poleward=xr.concat([freq_concat_merra2_fixed_poleward,freq_merra2_fixed_poleward],dim='time')
        
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    
    ds=xr.open_dataset(dir2+'era5.ar.labels.model.variable.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    ar=labeled/labeled
    freq_era5_polar=ar.sum(dim='time',skipna=True)
    
    ds=xr.open_dataset(dir2+'era5.ar.labels.model.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    ar=labeled/labeled
    freq_era5_fixed=ar.sum(dim='time',skipna=True)
    
    ds=xr.open_dataset(dir2+'era5.ar.labels.model.poleward.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    ar=labeled/labeled
    freq_era5_fixed_poleward=ar.sum(dim='time',skipna=True)
    
    if i==0:
        freq_concat_era5_polar=freq_era5_polar
        freq_concat_era5_fixed=freq_era5_fixed
        freq_concat_era5_fixed_poleward=freq_era5_fixed_poleward
    else:
        freq_concat_era5_polar=xr.concat([freq_concat_era5_polar,freq_era5_polar],dim='time')
        freq_concat_era5_fixed=xr.concat([freq_concat_era5_fixed,freq_era5_fixed],dim='time')
        freq_concat_era5_fixed_poleward=xr.concat([freq_concat_era5_fixed_poleward,freq_era5_fixed_poleward],dim='time')
    
    print(freq_concat_era5_polar.mean().values)
    print(freq_era5_polar.mean().values)

In [ ]:
mean1=freq_concat_merra2_polar.mean(dim='time',skipna=True).fillna(0)
mean2=freq_concat_merra2_fixed.mean(dim='time',skipna=True).fillna(0)
mean3=freq_concat_merra2_fixed_poleward.mean(dim='time',skipna=True).fillna(0)

mean4=freq_concat_era5_polar.mean(dim='time',skipna=True).fillna(0)
mean5=freq_concat_era5_fixed.mean(dim='time',skipna=True).fillna(0)
mean6=freq_concat_era5_fixed_poleward.mean(dim='time',skipna=True).fillna(0)

mean7=freq_concat_jra55_polar.mean(dim='time',skipna=True)
mean8=freq_concat_jra55_fixed.mean(dim='time',skipna=True)
mean9=freq_concat_jra55_fixed_poleward.mean(dim='time',skipna=True)

#print(mean1)
#print(mean7)
#sys.exit()

mean7=mean7.interp_like(mean1).fillna(0)
mean8=mean8.interp_like(mean1).fillna(0)
mean9=mean9.interp_like(mean1).fillna(0)



mean_all=(mean1+mean2+mean3+mean4+mean5+mean6+mean7+mean8+mean9)/9.

anomaly_merra2_polar=freq_concat_merra2_polar.mean(dim='time',skipna=True).fillna(0)-mean_all
anomaly_merra2_fixed=freq_concat_merra2_fixed.mean(dim='time',skipna=True).fillna(0)-mean_all
anomaly_merra2_fixed_poleward=freq_concat_merra2_fixed_poleward.mean(dim='time',skipna=True).fillna(0)-mean_all

anomaly_era5_polar=freq_concat_era5_polar.mean(dim='time',skipna=True).fillna(0)-mean_all
anomaly_era5_fixed=freq_concat_era5_fixed.mean(dim='time',skipna=True).fillna(0)-mean_all
anomaly_era5_fixed_poleward=freq_concat_era5_fixed_poleward.mean(dim='time',skipna=True).fillna(0)-mean_all

test1=freq_concat_jra55_polar.mean(dim='time',skipna=True).interp_like(mean1).fillna(0)
test2=freq_concat_jra55_fixed.mean(dim='time',skipna=True).interp_like(mean1).fillna(0)
test3=freq_concat_jra55_fixed_poleward.mean(dim='time',skipna=True).interp_like(mean1).fillna(0)

anomaly_jra55_polar=test1-mean_all
anomaly_jra55_fixed=test2-mean_all
anomaly_jra55_fixed_poleward=test3-mean_all

In [ ]:
print(field1)

In [ ]:
field=freq_concat_jra55_fixed_poleward.mean(dim='time',skipna=True)/365.25*100.
field1=mpcalc.smooth_n_point(anomaly_jra55_fixed_poleward,n=9,passes=20)/365.25*100.
field1[-10:,:]=mpcalc.smooth_n_point(anomaly_jra55_fixed_poleward[-10:,:],n=9,passes=1)/365.25*100.
field1[-2:,:]=anomaly_jra55_fixed_poleward[-2:,:]/365.25*100.

from matplotlib.colors import ListedColormap
from matplotlib.colors import LinearSegmentedColormap

cmap1=plt.cm.viridis(np.linspace(0,1,128))
cmap2=LinearSegmentedColormap.from_list('cmap2',[cmap1[-1],'orange'])
cmap2=cmap2(np.linspace(0,1,32))
colors=np.vstack((cmap1,cmap2))
cmap_new=LinearSegmentedColormap.from_list('cmap_new',colors)


fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())

#c=ax.contour(field.lon.values,field.lat.values,field/4.,levels=np.arange(20,240,40),colors='k',linewidths=2.0,alpha=0.96)
#plt.clabel(c,fmt='%.00f',fontsize=15,inline=True)

#cax=ax.pcolormesh(field.lon,field.lat,field/4.,vmin=0,vmax=130,cmap=plt.cm.viridis,transform=ccrs.PlateCarree(),alpha=1.0)   
cax=ax.contourf(field.lon,field.lat,field/4.,levels=np.arange(0,33,3),cmap=plt.cm.RdYlBu_r,extend='max',transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='max',aspect=25,ticks=np.arange(6,33,6))

c=ax.contour(field1.lon,field1.lat,field1/4.,levels=np.arange(-14,15.4,1.4),linewidths=4,colors='k',linestyle='solid')
plt.clabel(c,inline=True,fmt='%.01f',fontsize=22,inline_spacing=10)


g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=31) 
cbar.set_label('% time steps',fontsize=34,labelpad=5)
ax.coastlines(resolution='10m',color='w')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none',edgecolor='w')
ax.add_feature(countries)
g1.xlabel_style={'size':31,'color':'k'}
g1.ylabel_style={'size':31,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
#g1.left_labels=False
#g1.bottom_labels=False
#ax.set_title(r'$\bf{MERRA-2}$'+' '+r'$\bf{variable}$',fontsize=47,pad=5)
#ax.set_title(r'$\bf{MERRA-2}$'+' '+r'$\bf{fixed}$',fontsize=47,pad=5)
#ax.set_title(r'$\bf{MERRA-2}$'+' '+r'$\bf{fixed}$'+' '+r'$\bf{poleward}$',fontsize=47,pad=5)
#ax.set_title(r'$\bf{ERA5}$'+' '+r'$\bf{variable}$',fontsize=47,pad=5)
#ax.set_title(r'$\bf{ERA5}$'+' '+r'$\bf{fixed}$',fontsize=47,pad=5)
#ax.set_title(r'$\bf{ERA5}$'+' '+r'$\bf{fixed}$'+' '+r'$\bf{poleward}$',fontsize=47,pad=5)
#ax.set_title(r'$\bf{JRA-55}$'+' '+r'$\bf{variable}$',fontsize=47,pad=5)
#ax.set_title(r'$\bf{JRA-55}$'+' '+r'$\bf{fixed}$',fontsize=47,pad=5)
ax.set_title(r'$\bf{JRA-55}$'+' '+r'$\bf{fixed}$'+' '+r'$\bf{poleward}$',fontsize=47,pad=5)
plt.show()

In [ ]:
fig.savefig(dir_data+'ar_final_geomeans_9.png')

In [ ]:
lons_west=[x for x in freq_concat_jra55_polar.lon.values if x<-90]
lons_east=[x for x in freq_concat_jra55_polar.lon.values if x>=90]
sel_west=freq_concat_jra55_polar.sel(lon=lons_west)
sel_east=freq_concat_jra55_polar.sel(lon=lons_east)
sel_west['lon']=[x+360. for x in lons_west]
sel_east['lon']=[x-360. for x in lons_east]
final=xr.concat([sel_east,freq_concat_jra55_polar,sel_west],dim='lon')
interp=final.interp(lat=freq_concat_era5_polar.lat.values,lon=freq_concat_era5_polar.lon.values)
lons_center=[x for x in interp.lon.values if -180<=x<180]
freq_concat_jra55_polar1=interp.sel(lon=lons_center)


sel_west=freq_concat_jra55_fixed.sel(lon=lons_west)
sel_east=freq_concat_jra55_fixed.sel(lon=lons_east)
sel_west['lon']=[x+360. for x in lons_west]
sel_east['lon']=[x-360. for x in lons_east]
final=xr.concat([sel_east,freq_concat_jra55_fixed,sel_west],dim='lon')
interp=final.interp(lat=freq_concat_era5_polar.lat.values,lon=freq_concat_era5_polar.lon.values)
lons_center=[x for x in interp.lon.values if -180<=x<180]
freq_concat_jra55_fixed1=interp.sel(lon=lons_center)


sel_west=freq_concat_jra55_fixed_poleward.sel(lon=lons_west)
sel_east=freq_concat_jra55_fixed_poleward.sel(lon=lons_east)
sel_west['lon']=[x+360. for x in lons_west]
sel_east['lon']=[x-360. for x in lons_east]
final=xr.concat([sel_east,freq_concat_jra55_fixed_poleward,sel_west],dim='lon')
interp=final.interp(lat=freq_concat_era5_polar.lat.values,lon=freq_concat_era5_polar.lon.values)
lons_center=[x for x in interp.lon.values if -180<=x<180]
freq_concat_jra55_fixed_poleward1=interp.sel(lon=lons_center)

In [ ]:
dk=xr.Dataset()
dk['freq_concat_era5_variable']=(('time','lat','lon'),freq_concat_era5_polar.values)
dk['freq_concat_era5_fixed']=(('time','lat','lon'),freq_concat_era5_fixed.values)
dk['freq_concat_era5_fixed_poleward']=(('time','lat','lon'),freq_concat_era5_fixed_poleward.values)

dk['freq_concat_merra2_variable']=(('time','lat','lon'),freq_concat_merra2_polar.values)
dk['freq_concat_merra2_fixed']=(('time','lat','lon'),freq_concat_merra2_fixed.values)
dk['freq_concat_merra2_fixed_poleward']=(('time','lat','lon'),freq_concat_merra2_fixed_poleward.values)

dk['freq_concat_jra55_variable']=(('time','lat','lon'),freq_concat_jra55_polar1.values)
dk['freq_concat_jra55_fixed']=(('time','lat','lon'),freq_concat_jra55_fixed1.values)
dk['freq_concat_jra55_fixed_poleward']=(('time','lat','lon'),freq_concat_jra55_fixed_poleward1.values)

dk.coords['time']=yrs
dk.coords['lat']=freq_concat_era5_polar.lat.values
dk.coords['lon']=freq_concat_era5_polar.lon.values

dk['lat'].attrs["units"]='degrees_north'
dk['lon'].attrs["units"]='degrees_east'

try:
    os.remove(dir_data+'ar_freq_means_variants.nc')
except OSError:
    pass
dk.to_netcdf(dir_data+'ar_freq_means_variants.nc',mode='w',format='NETCDF4')

In [ ]:
ds=xr.open_dataset(dir_data+'ar_freq_means_variants.nc')
freq_concat_era5_polar=ds['freq_concat_era5_variable']
freq_concat_era5_fixed=ds['freq_concat_era5_fixed']
freq_concat_era5_fixed_poleward=ds['freq_concat_era5_fixed_poleward']

freq_concat_merra2_polar=ds['freq_concat_merra2_variable']
freq_concat_merra2_fixed=ds['freq_concat_merra2_fixed']
freq_concat_merra2_fixed_poleward=ds['freq_concat_merra2_fixed_poleward']

freq_concat_jra55_polar=ds['freq_concat_jra55_variable']
freq_concat_jra55_fixed=ds['freq_concat_jra55_fixed']
freq_concat_jra55_fixed_poleward=ds['freq_concat_jra55_fixed_poleward']

In [ ]:
print(freq_concat_era5_fixed[0,:,:].values)

In [ ]:
slopes_array_merra2_polar=xr.zeros_like(freq_concat_era5_polar[0,:,:])
sigs_array_merra2_polar=xr.zeros_like(freq_concat_era5_polar[0,:,:])
slopes_array_merra2_fixed=xr.zeros_like(freq_concat_era5_polar[0,:,:])
sigs_array_merra2_fixed=xr.zeros_like(freq_concat_era5_polar[0,:,:])
slopes_array_merra2_fixed_poleward=xr.zeros_like(freq_concat_era5_polar[0,:,:])
sigs_array_merra2_fixed_poleward=xr.zeros_like(freq_concat_era5_polar[0,:,:])

slopes_array_era5_polar=xr.zeros_like(freq_concat_era5_polar[0,:,:])
sigs_array_era5_polar=xr.zeros_like(freq_concat_era5_polar[0,:,:])
slopes_array_era5_fixed=xr.zeros_like(freq_concat_era5_polar[0,:,:])
sigs_array_era5_fixed=xr.zeros_like(freq_concat_era5_polar[0,:,:])
slopes_array_era5_fixed_poleward=xr.zeros_like(freq_concat_era5_polar[0,:,:])
sigs_array_era5_fixed_poleward=xr.zeros_like(freq_concat_era5_polar[0,:,:])

slopes_array_jra55_polar=xr.zeros_like(freq_concat_era5_polar[0,:,:])
sigs_array_jra55_polar=xr.zeros_like(freq_concat_era5_polar[0,:,:])
slopes_array_jra55_fixed=xr.zeros_like(freq_concat_era5_polar[0,:,:])
sigs_array_jra55_fixed=xr.zeros_like(freq_concat_era5_polar[0,:,:])
slopes_array_jra55_fixed_poleward=xr.zeros_like(freq_concat_era5_polar[0,:,:])
sigs_array_jra55_fixed_poleward=xr.zeros_like(freq_concat_era5_polar[0,:,:])
yrs=np.arange(1980,2024,1)
size=20
for j in range(freq_concat_merra2_polar.lat.size):
    print(j)
    for k in range(freq_concat_merra2_polar.lon.size):
        
        time_series=freq_concat_jra55_polar[:,j,k].values.tolist()
        if len([x for x in time_series if x>0])>=size:
            r=st.linregress(yrs,time_series)
            slopes_array_jra55_polar[j,k]=r[0]
            sigs_array_jra55_polar[j,k]=r[3]
        else:
            slopes_array_jra55_polar[j,k]=np.nan
            sigs_array_jra55_polar[j,k]=np.nan
        
        time_series=freq_concat_jra55_fixed[:,j,k].values.tolist()
        if len([x for x in time_series if x>0])>=size:
            r=st.linregress(yrs,time_series)
            slopes_array_jra55_fixed[j,k]=r[0]
            sigs_array_jra55_fixed[j,k]=r[3]
        else:
            slopes_array_jra55_fixed[j,k]=np.nan
            sigs_array_jra55_fixed[j,k]=np.nan
        
        time_series=freq_concat_jra55_fixed_poleward[:,j,k].values.tolist()
        if len([x for x in time_series if x>0])>=size:
            r=st.linregress(yrs,time_series)
            slopes_array_jra55_fixed_poleward[j,k]=r[0]
            sigs_array_jra55_fixed_poleward[j,k]=r[3]
        else:
            slopes_array_jra55_fixed_poleward[j,k]=np.nan
            sigs_array_jra55_fixed_poleward[j,k]=np.nan
        
        time_series=freq_concat_merra2_polar[:,j,k].values.tolist()
        if len([x for x in time_series if x>0])>=size:
            r=st.linregress(yrs,time_series)
            slopes_array_merra2_polar[j,k]=r[0]
            sigs_array_merra2_polar[j,k]=r[3]
        else:
            slopes_array_merra2_polar[j,k]=np.nan
            sigs_array_merra2_polar[j,k]=np.nan
        
        time_series=freq_concat_merra2_fixed[:,j,k].values.tolist()
        if len([x for x in time_series if x>0])>=size:
            r=st.linregress(yrs,time_series)
            slopes_array_merra2_fixed[j,k]=r[0]
            sigs_array_merra2_fixed[j,k]=r[3]
        else:
            slopes_array_merra2_fixed[j,k]=np.nan
            sigs_array_merra2_fixed[j,k]=np.nan
        
        time_series=freq_concat_merra2_fixed_poleward[:,j,k].values.tolist()
        if len([x for x in time_series if x>0])>=size:
            r=st.linregress(yrs,time_series)
            slopes_array_merra2_fixed_poleward[j,k]=r[0]
            sigs_array_merra2_fixed_poleward[j,k]=r[3]
        else:
            slopes_array_merra2_fixed_poleward[j,k]=np.nan
            sigs_array_merra2_fixed_poleward[j,k]=np.nan
        
        time_series=freq_concat_era5_polar[:,j,k].values.tolist()
        if len([x for x in time_series if x>0])>=size:
            r=st.linregress(yrs,time_series)
            slopes_array_era5_polar[j,k]=r[0]
            sigs_array_era5_polar[j,k]=r[3]
        else:
            slopes_array_era5_polar[j,k]=np.nan
            sigs_array_era5_polar[j,k]=np.nan
        
        time_series=freq_concat_era5_fixed[:,j,k].values.tolist()
        if len([x for x in time_series if x>0])>=size:
            r=st.linregress(yrs,time_series)
            slopes_array_era5_fixed[j,k]=r[0]
            sigs_array_era5_fixed[j,k]=r[3]
        else:
            slopes_array_era5_fixed[j,k]=np.nan
            sigs_array_era5_fixed[j,k]=np.nan
        
        time_series=freq_concat_era5_fixed_poleward[:,j,k].values.tolist()
        if len([x for x in time_series if x>0])>=size:
            r=st.linregress(yrs,time_series)
            slopes_array_era5_fixed_poleward[j,k]=r[0]
            sigs_array_era5_fixed_poleward[j,k]=r[3]
        else:
            slopes_array_era5_fixed_poleward[j,k]=np.nan
            sigs_array_era5_fixed_poleward[j,k]=np.nan

In [ ]:
dk=xr.Dataset()
dk['slopes_array_era5_variable']=(('lat','lon'),slopes_array_era5_polar.values)
dk['slopes_array_era5_fixed']=(('lat','lon'),slopes_array_era5_fixed.values)
dk['slopes_array_era5_fixed_poleward']=(('lat','lon'),slopes_array_era5_fixed_poleward.values)

dk['slopes_array_jra55_variable']=(('lat','lon'),slopes_array_jra55_polar.values)
dk['slopes_array_jra55_fixed']=(('lat','lon'),slopes_array_jra55_fixed.values)
dk['slopes_array_jra55_fixed_poleward']=(('lat','lon'),slopes_array_jra55_fixed_poleward.values)

dk['slopes_array_merra2_variable']=(('lat','lon'),slopes_array_merra2_polar.values)
dk['slopes_array_merra2_fixed']=(('lat','lon'),slopes_array_merra2_fixed.values)
dk['slopes_array_merra2_fixed_poleward']=(('lat','lon'),slopes_array_merra2_fixed_poleward.values)

dk['sigs_array_era5_variable']=(('lat','lon'),sigs_array_era5_polar.values)
dk['sigs_array_era5_fixed']=(('lat','lon'),sigs_array_era5_fixed.values)
dk['sigs_array_era5_fixed_poleward']=(('lat','lon'),sigs_array_era5_fixed_poleward.values)

dk['sigs_array_jra55_variable']=(('lat','lon'),sigs_array_jra55_polar.values)
dk['sigs_array_jra55_fixed']=(('lat','lon'),sigs_array_jra55_fixed.values)
dk['sigs_array_jra55_fixed_poleward']=(('lat','lon'),sigs_array_jra55_fixed_poleward.values)

dk['sigs_array_merra2_variable']=(('lat','lon'),sigs_array_merra2_polar.values)
dk['sigs_array_merra2_fixed']=(('lat','lon'),sigs_array_merra2_fixed.values)
dk['sigs_array_merra2_fixed_poleward']=(('lat','lon'),sigs_array_merra2_fixed_poleward.values)

dk.coords['lat']=slopes_array_era5_polar.lat.values
dk.coords['lon']=slopes_array_era5_polar.lon.values

dk['lat'].attrs["units"]='degrees_north'
dk['lon'].attrs["units"]='degrees_east'

try:
    os.remove(dir_data+'ar_geoslopes_variants.nc')
except OSError:
    pass
dk.to_netcdf(dir_data+'ar_geoslopes_variants.nc',mode='w',format='NETCDF4')

In [ ]:
N=sigs_array_merra2_polar.lat.size*sigs_array_merra2_polar.lon.size
a=0.05#/sum_factor

p_values=sorted(sigs_array_merra2_polar.values.flatten().tolist())
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)

print(N)
print(len([x for x in p_values if x>=0]))

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final=max(pass_list)
print(p_final)
sigs_array_final_merra2_polar=sigs_array_merra2_polar.where(sigs_array_merra2_polar<=p_final)


p_values=sorted(sigs_array_merra2_fixed.values.flatten().tolist())
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final=max(pass_list)
print(p_final)
sigs_array_final_merra2_fixed=sigs_array_merra2_fixed.where(sigs_array_merra2_fixed<=p_final)


p_values=sorted(sigs_array_merra2_fixed_poleward.values.flatten().tolist())
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final=max(pass_list)
print(p_final)
sigs_array_final_merra2_fixed_poleward=sigs_array_merra2_fixed_poleward.where(sigs_array_merra2_fixed_poleward<=p_final)

########

N=sigs_array_era5_polar.lat.size*sigs_array_era5_polar.lon.size
p_values=sorted(sigs_array_era5_polar.values.flatten().tolist())
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)

print(N)
print(len([x for x in p_values if x>=0]))

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
except:
    p_final=0
print(p_final)
sigs_array_final_era5_polar=sigs_array_era5_polar.where(sigs_array_era5_polar<=p_final)


p_values=sorted(sigs_array_era5_fixed.values.flatten().tolist())
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
except:
    p_final=0
print(p_final)
sigs_array_final_era5_fixed=sigs_array_era5_fixed.where(sigs_array_era5_fixed<=p_final)


p_values=sorted(sigs_array_era5_fixed_poleward.values.flatten().tolist())
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
except:
    p_final=0
print(p_final)
sigs_array_final_era5_fixed_poleward=sigs_array_era5_fixed_poleward.where(sigs_array_era5_fixed_poleward<=p_final)

############

N=sigs_array_jra55_polar.lat.size*sigs_array_jra55_polar.lon.size
p_values=sorted(sigs_array_jra55_polar.values.flatten().tolist())
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)

print(N)
print(len([x for x in p_values if x>=0]))

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
except:
    p_final=0
print(p_final)
sigs_array_final_jra55_polar=sigs_array_jra55_polar.where(sigs_array_jra55_polar<=p_final)


p_values=sorted(sigs_array_jra55_fixed.values.flatten().tolist())
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
except:
    p_final=0
print(p_final)
sigs_array_final_jra55_fixed=sigs_array_jra55_fixed.where(sigs_array_jra55_fixed<=p_final)


p_values=sorted(sigs_array_jra55_fixed_poleward.values.flatten().tolist())
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
except:
    p_final=0
print(p_final)
sigs_array_final_jra55_fixed_poleward=sigs_array_jra55_fixed_poleward.where(sigs_array_jra55_fixed_poleward<=p_final)

In [ ]:
num_sets1=xr.ones_like(slopes_array_merra2_polar).where(slopes_array_merra2_polar>-1000)
num_sets1=num_sets1.fillna(0)
num_sets2=xr.ones_like(slopes_array_merra2_fixed).where(slopes_array_merra2_fixed>-1000)
num_sets2=num_sets2.fillna(0)
num_sets3=xr.ones_like(slopes_array_merra2_fixed_poleward).where(slopes_array_merra2_fixed_poleward>-1000)
num_sets3=num_sets3.fillna(0)

num_sets4=xr.ones_like(slopes_array_era5_polar).where(slopes_array_era5_polar>-1000)
num_sets4=num_sets4.fillna(0)
num_sets5=xr.ones_like(slopes_array_era5_fixed).where(slopes_array_era5_fixed>-1000)
num_sets5=num_sets5.fillna(0)
num_sets6=xr.ones_like(slopes_array_era5_fixed_poleward).where(slopes_array_era5_fixed_poleward>-1000)
num_sets6=num_sets6.fillna(0)

num_sets7=xr.ones_like(slopes_array_jra55_polar).where(slopes_array_jra55_polar>-1000)
num_sets7=num_sets7.fillna(0)
num_sets8=xr.ones_like(slopes_array_jra55_fixed).where(slopes_array_jra55_fixed>-1000)
num_sets8=num_sets8.fillna(0)
num_sets9=xr.ones_like(slopes_array_jra55_fixed_poleward).where(slopes_array_jra55_fixed_poleward>-1000)
num_sets9=num_sets9.fillna(0)

num_sets=num_sets1+num_sets2+num_sets3+num_sets4+num_sets5+num_sets6+num_sets7+num_sets8+num_sets9
field=num_sets


fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field.where(field>0),levels=np.arange(0,10,1),cmap=plt.cm.Greens,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(1,10,1))

#cax=ax.contourf(field.lon,field.lat,field,levels=np.arange(-2,2.2,0.2),cmap=plt.cm.seismic,extend='both',transform=ccrs.PlateCarree(),alpha=1.0)   


g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=32) 
cbar.set_label('# datasets',fontsize=36,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':28,'color':'k'}
g1.ylabel_style={'size':28,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.left_labels=False
#g1.bottom_labels=False
#ax.set_ylim(-60,70)

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
ax.set_title(r'$\bf{\#}$'+' '+r'$\bf{datasets:}$'+' '+r'$\bf{3}$'+' '+r'$\bf{AR}$'+' '+r'$\bf{variants}$',fontsize=40,pad=5)

plt.show()

In [ ]:
#fig.savefig(dir_data+'ar_trend_datasetsa_new.png')
fig.savefig(dir_data+'artmip_redo_3.png')

In [ ]:
ds=xr.open_dataset(dir_data+'merra2_gridareas.nc')
areas=ds['cell_area']
test1=areas.where(test_all>=66.6).sum()
test2=areas.sum()
ratio=test1/test2
print(ratio)

In [ ]:
test1=xr.ones_like(sigs_array_final_merra2_polar).where(sigs_array_final_merra2_polar<=0.05).where(slopes_array_merra2_polar>0).fillna(0)
test2=xr.ones_like(sigs_array_final_merra2_fixed).where(sigs_array_final_merra2_fixed<=0.05).where(slopes_array_merra2_fixed>0).fillna(0)
test3=xr.ones_like(sigs_array_final_merra2_polar).where(sigs_array_final_merra2_fixed_poleward<=0.05).where(slopes_array_merra2_fixed_poleward>0).fillna(0)
test4=xr.ones_like(sigs_array_final_era5_polar).where(sigs_array_final_era5_polar<=0.05).where(slopes_array_era5_polar>0).fillna(0)
test5=xr.ones_like(sigs_array_final_era5_polar).where(sigs_array_final_era5_fixed<=0.05).where(slopes_array_era5_fixed>0).fillna(0)
test6=xr.ones_like(sigs_array_final_era5_polar).where(sigs_array_final_era5_fixed_poleward<=0.05).where(slopes_array_era5_fixed_poleward>0).fillna(0)
test7=xr.ones_like(sigs_array_final_jra55_polar).where(sigs_array_final_jra55_polar<=0.05).where(slopes_array_jra55_polar>0).fillna(0)
test8=xr.ones_like(sigs_array_final_jra55_polar).where(sigs_array_final_jra55_fixed<=0.05).where(slopes_array_jra55_fixed>0).fillna(0)
test9=xr.ones_like(sigs_array_final_jra55_polar).where(sigs_array_final_jra55_fixed_poleward<=0.05).where(slopes_array_jra55_fixed_poleward>0).fillna(0)

test_all=(test1+test2+test3+test4+test5+test6+test7+test8+test9)/num_sets*100.
field=test_all

test1a=xr.ones_like(sigs_array_final_merra2_polar).where(sigs_array_final_merra2_polar<=0.05).where(slopes_array_merra2_polar<0).fillna(0)
test2a=xr.ones_like(sigs_array_final_merra2_fixed).where(sigs_array_final_merra2_fixed<=0.05).where(slopes_array_merra2_fixed<0).fillna(0)
test3a=xr.ones_like(sigs_array_final_merra2_polar).where(sigs_array_final_merra2_fixed_poleward<=0.05).where(slopes_array_merra2_fixed_poleward<0).fillna(0)
test4a=xr.ones_like(sigs_array_final_era5_polar).where(sigs_array_final_era5_polar<=0.05).where(slopes_array_era5_polar<0).fillna(0)
test5a=xr.ones_like(sigs_array_final_era5_polar).where(sigs_array_final_era5_fixed<=0.05).where(slopes_array_era5_fixed<0).fillna(0)
test6a=xr.ones_like(sigs_array_final_era5_polar).where(sigs_array_final_era5_fixed_poleward<=0.05).where(slopes_array_era5_fixed_poleward<0).fillna(0)
test7a=xr.ones_like(sigs_array_final_jra55_polar).where(sigs_array_final_jra55_polar<=0.05).where(slopes_array_jra55_polar<0).fillna(0)
test8a=xr.ones_like(sigs_array_final_jra55_polar).where(sigs_array_final_jra55_fixed<=0.05).where(slopes_array_jra55_fixed<0).fillna(0)
test9a=xr.ones_like(sigs_array_final_jra55_polar).where(sigs_array_final_jra55_fixed_poleward<=0.05).where(slopes_array_jra55_fixed_poleward<0).fillna(0)

test_alla=(test1a+test2a+test3a+test4a+test5a+test6a+test7a+test8a+test9a)/num_sets*100.
fielda=test_alla

fieldb=field.where(field==fielda)
fieldb=fieldb/fieldb
field=field.where(field>fielda).fillna(0)
fielda=fielda.where(fielda>field)

####Lenient

test1b=xr.ones_like(sigs_array_final_merra2_polar).where(sigs_array_merra2_polar<=0.05).where(slopes_array_merra2_polar>0).fillna(0)
test2b=xr.ones_like(sigs_array_final_merra2_fixed).where(sigs_array_merra2_fixed<=0.05).where(slopes_array_merra2_fixed>0).fillna(0)
test3b=xr.ones_like(sigs_array_final_merra2_polar).where(sigs_array_merra2_fixed_poleward<=0.05).where(slopes_array_merra2_fixed_poleward>0).fillna(0)
test4b=xr.ones_like(sigs_array_final_era5_polar).where(sigs_array_era5_polar<=0.05).where(slopes_array_era5_polar>0).fillna(0)
test5b=xr.ones_like(sigs_array_final_era5_polar).where(sigs_array_era5_fixed<=0.05).where(slopes_array_era5_fixed>0).fillna(0)
test6b=xr.ones_like(sigs_array_final_era5_polar).where(sigs_array_era5_fixed_poleward<=0.05).where(slopes_array_era5_fixed_poleward>0).fillna(0)
test7b=xr.ones_like(sigs_array_final_jra55_polar).where(sigs_array_jra55_polar<=0.05).where(slopes_array_jra55_polar>0).fillna(0)
test8b=xr.ones_like(sigs_array_final_jra55_polar).where(sigs_array_jra55_fixed<=0.05).where(slopes_array_jra55_fixed>0).fillna(0)
test9b=xr.ones_like(sigs_array_final_jra55_polar).where(sigs_array_jra55_fixed_poleward<=0.05).where(slopes_array_jra55_fixed_poleward>0).fillna(0)

test_allb=(test1b+test2b+test3b+test4b+test5b+test6b+test7b+test8b+test9b)/num_sets*100.

test1c=xr.ones_like(sigs_array_final_merra2_polar).where(sigs_array_merra2_polar<=0.05).where(slopes_array_merra2_polar<0).fillna(0)
test2c=xr.ones_like(sigs_array_final_merra2_fixed).where(sigs_array_merra2_fixed<=0.05).where(slopes_array_merra2_fixed<0).fillna(0)
test3c=xr.ones_like(sigs_array_final_merra2_polar).where(sigs_array_merra2_fixed_poleward<=0.05).where(slopes_array_merra2_fixed_poleward<0).fillna(0)
test4c=xr.ones_like(sigs_array_final_era5_polar).where(sigs_array_era5_polar<=0.05).where(slopes_array_era5_polar<0).fillna(0)
test5c=xr.ones_like(sigs_array_final_era5_polar).where(sigs_array_era5_fixed<=0.05).where(slopes_array_era5_fixed<0).fillna(0)
test6c=xr.ones_like(sigs_array_final_era5_polar).where(sigs_array_era5_fixed_poleward<=0.05).where(slopes_array_era5_fixed_poleward<0).fillna(0)
test7c=xr.ones_like(sigs_array_final_jra55_polar).where(sigs_array_jra55_polar<=0.05).where(slopes_array_jra55_polar<0).fillna(0)
test8c=xr.ones_like(sigs_array_final_jra55_polar).where(sigs_array_jra55_fixed<=0.05).where(slopes_array_jra55_fixed<0).fillna(0)
test9c=xr.ones_like(sigs_array_final_jra55_polar).where(sigs_array_jra55_fixed_poleward<=0.05).where(slopes_array_jra55_fixed_poleward<0).fillna(0)

test_allc=(test1c+test2c+test3c+test4c+test5c+test6c+test7c+test8c+test9c)/num_sets*100.

from matplotlib.colors import LinearSegmentedColormap
colors1=plt.cm.Reds(np.linspace(0,1,128))
colors2=plt.cm.Blues_r(np.linspace(0,1,128))
colors=np.vstack((colors2,colors1))
cmap_new=LinearSegmentedColormap.from_list('cmap_new',colors)

fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field.where(field>0),levels=np.arange(-100,110,10),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(-100,120,20))

cax=ax.contourf(field.lon,field.lat,fielda.where(fielda>0)*-1.,levels=np.arange(-100,110,10),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(1,10,1))
cbar.ax.set_yticklabels([' ','80','60','40','20','0','20','40','60','80',' '])

cmap_new=LinearSegmentedColormap.from_list('cmap_new',['w','magenta'])
cax=ax.pcolormesh(fieldb.lon,fieldb.lat,fieldb,vmin=0,vmax=1.0,cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   

X=4
for j in range(int(field.lat.size/X)):
    for k in range(int(field.lon.size/X)):
        if test_allb[X*j,X*k]>=66.66 or test_allc[X*j,X*k]>=66.66:
            ax.plot(field.lon[X*k],field.lat[X*j],marker='o',color='k',markersize=2,alpha=0.5)

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=32) 
cbar.set_label('% datasets',fontsize=36,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':28,'color':'k'}
g1.ylabel_style={'size':28,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.left_labels=False
#g1.bottom_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
ax.set_title(r'$\bf{Trends:}$'+' '+r'$\bf{3}$'+' '+r'$\bf{AR}$'+' '+r'$\bf{variants}$',fontsize=40,pad=5)

plt.show()

In [ ]:
#fig.savefig(dir_data+'ar_final_summary_new.png')
fig.savefig(dir_data+'artmip_redo_5.png')

In [ ]:
field=slopes_array_jra55_fixed_poleward
field1=sigs_array_final_jra55_fixed_poleward
ones=freq_concat_jra55_fixed_poleward/freq_concat_jra55_fixed_poleward
mean=freq_concat_jra55_fixed_poleward.mean(dim='time')

ones_sum=ones.sum(dim='time',skipna=True)
field=field/mean*100.
field=field.where(ones_sum>=43*0.9)
field1=field1.where(ones_sum>=43*0.9)

from matplotlib.colors import ListedColormap
cmap1=plt.cm.seismic(np.linspace(0.2,0.8,128))
cmap_new=ListedColormap(cmap1)

fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.pcolormesh(field.lon,field.lat,field,vmin=-1.5,vmax=1.5,cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='both',aspect=25,ticks=np.arange(-1.5,2,0.5))

#X=6
X=4
for j in range(int(field1.lat.size/X)):
    for k in range(int(field1.lon.size/X)):
        if field1[X*j,X*k].values.tolist()<=0.05:
            ax.plot(field.lon.values[X*k],field.lat.values[X*j],marker='o',color='yellow',markersize=3)
            #ax.plot(field.lon.values[X*k],field.lat.values[X*j],marker='o',color='yellow',markersize=3.5)
        #elif field1[X*j,X*k].values.tolist()<=0.05:
        #    ax.plot(field.lon.values[X*k],field.lat.values[X*j],marker='o',color='w',markersize=1.75)#1.5

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=31) 
cbar.set_label('% yr$^{-1}$',fontsize=34,labelpad=5)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':31,'color':'k'}
g1.ylabel_style={'size':31,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.left_labels=False
#g1.bottom_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
#ax.set_title(r'$\bf{MERRA-2}$'+' '+r'$\bf{variable}$',fontsize=47,pad=5)
#ax.set_title(r'$\bf{MERRA-2}$'+' '+r'$\bf{fixed}$',fontsize=47,pad=5)
#ax.set_title(r'$\bf{MERRA-2}$'+' '+r'$\bf{fixed}$'+' '+r'$\bf{poleward}$',fontsize=47,pad=5)
#ax.set_title(r'$\bf{ERA5}$'+' '+r'$\bf{variable}$',fontsize=47,pad=5)
#ax.set_title(r'$\bf{ERA5}$'+' '+r'$\bf{fixed}$',fontsize=47,pad=5)
#ax.set_title(r'$\bf{ERA5}$'+' '+r'$\bf{fixed}$'+' '+r'$\bf{poleward}$',fontsize=47,pad=5)
#ax.set_title(r'$\bf{JRA-55}$'+' '+r'$\bf{variable}$',fontsize=47,pad=5)
#ax.set_title(r'$\bf{JRA-55}$'+' '+r'$\bf{fixed}$',fontsize=47,pad=5)
ax.set_title(r'$\bf{JRA-55}$'+' '+r'$\bf{fixed}$'+' '+r'$\bf{poleward}$',fontsize=47,pad=5)

plt.show()

In [ ]:
fig.savefig(dir_data+'ar_final_geotrends_9.png')